In [1]:
import os
import pandas as pd
import numpy as np
import sqlite3

## Creating the pipeData

In [2]:
# Load the CSV
TeAllPointsFile = "../resources/TeAllPoints.csv"
pointsdf = pd.read_csv(TeAllPointsFile, encoding="ISO-8859-1")

# Loc column has leading spaces, so we convert to remove them
pointsdf.Loc = pd.to_numeric(pointsdf['Loc'], errors='coerce')

# pick the columns for the table
pipeDataTable = pd.DataFrame({
    'loc': pointsdf['Loc'],
    'locName': pointsdf['Loc Name'],
    'locState': pointsdf['Loc St Abbrev'],
    'locCounty': pointsdf['Loc Cnty'],
    'locZone': pointsdf['Loc Zone'],
    'dirFlo': pointsdf['Dir Flo'],
    'locType': pointsdf['Loc Type Ind']
})

pipeDataTable.head(1)

,loc,locName,locState,locCounty,locZone,dirFlo,locType
0,70059,"Chevron - Perth Amboy, NJ",NJ,MIDDLESEX,3,Delivery,UNK


## Creating the dailyFlo

In [4]:
# Load the XLSX
flowsFile = "../resources/flows_last10.xlsx"
flowsdf = pd.concat(pd.read_excel(flowsFile, sheet_name=None), ignore_index=True)

dailyFloTable = pd.DataFrame({
    'loc': flowsdf.Loc,
    'gas_date': flowsdf.Eff_Gas_Day,
    'Total_Design_Capacity': flowsdf.Total_Design_Capacity,
    'Operating_Capacity': flowsdf.Operating_Capacity,
    'Total_Scheduled_Quantity': flowsdf.Total_Scheduled_Quantity,
    'TSP_Name': flowsdf.TSP_Name
})

dailyFloTable.head(1)

,loc,gas_date,Total_Design_Capacity,Operating_Capacity,Total_Scheduled_Quantity,TSP_Name
0,70004,2020-08-21,593830,600490,0,TX EAST TRAN


## Combining the dataframes for the DB

In [5]:
finalframe = pd.merge(dailyFloTable, pipeDataTable, on='loc', how='inner')
finalframe.head(1)

,loc,gas_date,Total_Design_Capacity,Operating_Capacity,Total_Scheduled_Quantity,TSP_Name,locName,locState,locCounty,locZone,dirFlo,locType
0,70004,2020-08-21,593830,600490,0,TX EAST TRAN,"DOMINION ENERGY TRANSMISSION, INC - CLARINGTON...",OH,MONROE,2,D,INT


## Sending data to the DB

In [10]:
conn = sqlite3.connect('ngdbp2.sqlite')
c = conn.cursor()

In [11]:
c.execute('''CREATE TABLE dailyFlo
            ([loc] TEXT,
            [gas_date] TEXT,
            [Total_Design_Capacity] TEXT,
            [Operating_Capacity] TEXT,
            [Total_Scheduled_Quantity] TEXT,
            [TSP_Name] TEXT,
            [locName] TEXT,
            [locState] TEXT,
            [locCounty] TEXT,
            [locZone] TEXT,
            [dirFlo] TEXT,
            [locType] TEXT,
            PRIMARY KEY(loc,gas_date))''')

In [12]:
finalframe.to_sql('dailyFlo', conn, if_exists='append', index = False)

In [13]:
c.execute("SELECT * FROM dailyFlo")

rows = c.fetchall()

for row in rows:
    print(row)

('70004', '2020-08-21 00:00:00', '593830', '600490', '0', 'TX EAST TRAN', 'DOMINION ENERGY TRANSMISSION, INC - CLARINGTON, OH', 'OH', 'MONROE', '2', 'D', 'INT')
('70004', '2020-08-22 00:00:00', '593830', '600490', '0', 'TX EAST TRAN', 'DOMINION ENERGY TRANSMISSION, INC - CLARINGTON, OH', 'OH', 'MONROE', '2', 'D', 'INT')
('70004', '2020-08-23 00:00:00', '593830', '600490', '0', 'TX EAST TRAN', 'DOMINION ENERGY TRANSMISSION, INC - CLARINGTON, OH', 'OH', 'MONROE', '2', 'D', 'INT')
('70004', '2020-08-24 00:00:00', '593830', '600490', '0', 'TX EAST TRAN', 'DOMINION ENERGY TRANSMISSION, INC - CLARINGTON, OH', 'OH', 'MONROE', '2', 'D', 'INT')
('70004', '2020-08-25 00:00:00', '593830', '600490', '0', 'TX EAST TRAN', 'DOMINION ENERGY TRANSMISSION, INC - CLARINGTON, OH', 'OH', 'MONROE', '2', 'D', 'INT')
('70004', '2020-08-26 00:00:00', '593830', '600490', '0', 'TX EAST TRAN', 'DOMINION ENERGY TRANSMISSION, INC - CLARINGTON, OH', 'OH', 'MONROE', '2', 'D', 'INT')
('70004', '2020-08-27 00:00:00', '

('71000', '2020-08-30 00:00:00', '243324', '243324', '49270', 'TX EAST TRAN', 'GULF SOUTH PIPELINE COMPANY, LP - WEST MONROE, LA  (RECEIPT', 'LA', 'OUACHITA', 'ELA', 'R', 'INT')
('71020', '2020-08-21 00:00:00', '944', '944', '1', 'TX EAST TRAN', 'TE PRINCETON - PRINCETON, IN', 'IN', 'GIBSON', '2', 'D', 'LDC')
('71020', '2020-08-22 00:00:00', '944', '944', '1', 'TX EAST TRAN', 'TE PRINCETON - PRINCETON, IN', 'IN', 'GIBSON', '2', 'D', 'LDC')
('71020', '2020-08-23 00:00:00', '944', '944', '1', 'TX EAST TRAN', 'TE PRINCETON - PRINCETON, IN', 'IN', 'GIBSON', '2', 'D', 'LDC')
('71020', '2020-08-24 00:00:00', '944', '944', '1', 'TX EAST TRAN', 'TE PRINCETON - PRINCETON, IN', 'IN', 'GIBSON', '2', 'D', 'LDC')
('71020', '2020-08-25 00:00:00', '944', '944', '1', 'TX EAST TRAN', 'TE PRINCETON - PRINCETON, IN', 'IN', 'GIBSON', '2', 'D', 'LDC')
('71020', '2020-08-26 00:00:00', '944', '944', '1', 'TX EAST TRAN', 'TE PRINCETON - PRINCETON, IN', 'IN', 'GIBSON', '2', 'D', 'LDC')
('71020', '2020-08-27 00

('71439', '2020-08-29 00:00:00', '5878', '5878', '325', 'TX EAST TRAN', 'INDIANA GAS CO. - DUBOIS, IN', 'IN', 'DUBOIS', '2', 'D', 'LDC')
('71439', '2020-08-30 00:00:00', '5878', '5878', '285', 'TX EAST TRAN', 'INDIANA GAS CO. - DUBOIS, IN', 'IN', 'DUBOIS', '2', 'D', 'LDC')
('71455', '2020-08-21 00:00:00', '5323', '5353', '0', 'TX EAST TRAN', 'UGI CENTRAL - HUNTINGDON GAS - MAPLETON, PA', 'PA', 'HUNTINGDON', '3', 'D', 'LDC')
('71455', '2020-08-22 00:00:00', '5323', '5353', '0', 'TX EAST TRAN', 'UGI CENTRAL - HUNTINGDON GAS - MAPLETON, PA', 'PA', 'HUNTINGDON', '3', 'D', 'LDC')
('71455', '2020-08-23 00:00:00', '5323', '5353', '0', 'TX EAST TRAN', 'UGI CENTRAL - HUNTINGDON GAS - MAPLETON, PA', 'PA', 'HUNTINGDON', '3', 'D', 'LDC')
('71455', '2020-08-24 00:00:00', '5323', '5353', '0', 'TX EAST TRAN', 'UGI CENTRAL - HUNTINGDON GAS - MAPLETON, PA', 'PA', 'HUNTINGDON', '3', 'D', 'LDC')
('71455', '2020-08-25 00:00:00', '5323', '5353', '0', 'TX EAST TRAN', 'UGI CENTRAL - HUNTINGDON GAS - MAPLETON

('72603', '2020-08-25 00:00:00', '131284', '131284', '10000', 'TX EAST TRAN', 'KINDER MORGAN TEJAS (FROM TE) UPRIVER RD., NUECES CO', 'TX', 'NUECES', 'STX', 'D', 'INT')
('72603', '2020-08-26 00:00:00', '131284', '131284', '10000', 'TX EAST TRAN', 'KINDER MORGAN TEJAS (FROM TE) UPRIVER RD., NUECES CO', 'TX', 'NUECES', 'STX', 'D', 'INT')
('72603', '2020-08-27 00:00:00', '131284', '131284', '5000', 'TX EAST TRAN', 'KINDER MORGAN TEJAS (FROM TE) UPRIVER RD., NUECES CO', 'TX', 'NUECES', 'STX', 'D', 'INT')
('72603', '2020-08-28 00:00:00', '131284', '131284', '5000', 'TX EAST TRAN', 'KINDER MORGAN TEJAS (FROM TE) UPRIVER RD., NUECES CO', 'TX', 'NUECES', 'STX', 'D', 'INT')
('72603', '2020-08-29 00:00:00', '131284', '131284', '5000', 'TX EAST TRAN', 'KINDER MORGAN TEJAS (FROM TE) UPRIVER RD., NUECES CO', 'TX', 'NUECES', 'STX', 'D', 'INT')
('72603', '2020-08-30 00:00:00', '131284', '131284', '5000', 'TX EAST TRAN', 'KINDER MORGAN TEJAS (FROM TE) UPRIVER RD., NUECES CO', 'TX', 'NUECES', 'STX', 'D

('72950', '2020-08-28 00:00:00', '4092', '4092', '382', 'TX EAST TRAN', 'APTIAN GAS CORP., K.W. SCHREINER #1 - LIVE OAK CO., TX', 'TX', 'LIVE OAK', 'STX', 'R', 'WHD')
('72950', '2020-08-29 00:00:00', '4092', '4092', '382', 'TX EAST TRAN', 'APTIAN GAS CORP., K.W. SCHREINER #1 - LIVE OAK CO., TX', 'TX', 'LIVE OAK', 'STX', 'R', 'WHD')
('72950', '2020-08-30 00:00:00', '4092', '4092', '382', 'TX EAST TRAN', 'APTIAN GAS CORP., K.W. SCHREINER #1 - LIVE OAK CO., TX', 'TX', 'LIVE OAK', 'STX', 'R', 'WHD')
('72956', '2020-08-21 00:00:00', '16248', '16248', '0', 'TX EAST TRAN', 'MID TN GAS DIST - WILSON, TN', 'TN', 'WILSON', '1', 'D', 'LDC')
('72956', '2020-08-22 00:00:00', '16248', '16248', '0', 'TX EAST TRAN', 'MID TN GAS DIST - WILSON, TN', 'TN', 'WILSON', '1', 'D', 'LDC')
('72956', '2020-08-23 00:00:00', '16248', '16248', '0', 'TX EAST TRAN', 'MID TN GAS DIST - WILSON, TN', 'TN', 'WILSON', '1', 'D', 'LDC')
('72956', '2020-08-24 00:00:00', '16248', '16248', '0', 'TX EAST TRAN', 'MID TN GAS DIST

('73195', '2020-08-27 00:00:00', '214278', '214278', '51185', 'TX EAST TRAN', 'COL. GAS - MONTGOMERY CO., KY', 'KY', 'MONTGOMERY', '2', 'R', 'INT')
('73195', '2020-08-28 00:00:00', '214278', '214278', '51245', 'TX EAST TRAN', 'COL. GAS - MONTGOMERY CO., KY', 'KY', 'MONTGOMERY', '2', 'R', 'INT')
('73195', '2020-08-29 00:00:00', '214278', '214278', '51140', 'TX EAST TRAN', 'COL. GAS - MONTGOMERY CO., KY', 'KY', 'MONTGOMERY', '2', 'R', 'INT')
('73195', '2020-08-30 00:00:00', '214278', '214278', '51245', 'TX EAST TRAN', 'COL. GAS - MONTGOMERY CO., KY', 'KY', 'MONTGOMERY', '2', 'R', 'INT')
('73196', '2020-08-21 00:00:00', '4310', '4310', '5', 'TX EAST TRAN', 'DELTA NAT GAS -MADISON CO., KY', 'KY', 'MADISON', '2', 'D', 'LDC')
('73196', '2020-08-22 00:00:00', '4310', '4310', '5', 'TX EAST TRAN', 'DELTA NAT GAS -MADISON CO., KY', 'KY', 'MADISON', '2', 'D', 'LDC')
('73196', '2020-08-23 00:00:00', '4310', '4310', '5', 'TX EAST TRAN', 'DELTA NAT GAS -MADISON CO., KY', 'KY', 'MADISON', '2', 'D', '

('73412', '2020-08-27 00:00:00', '4038', '4038', '0', 'TX EAST TRAN', 'BUCKEYE RANCH GATHERING - MATAGORDA CO., TX', 'TX', 'MATAGORDA', 'STX', 'R', 'WHD')
('73412', '2020-08-28 00:00:00', '4038', '4038', '0', 'TX EAST TRAN', 'BUCKEYE RANCH GATHERING - MATAGORDA CO., TX', 'TX', 'MATAGORDA', 'STX', 'R', 'WHD')
('73412', '2020-08-29 00:00:00', '4038', '4038', '0', 'TX EAST TRAN', 'BUCKEYE RANCH GATHERING - MATAGORDA CO., TX', 'TX', 'MATAGORDA', 'STX', 'R', 'WHD')
('73412', '2020-08-30 00:00:00', '4038', '4038', '0', 'TX EAST TRAN', 'BUCKEYE RANCH GATHERING - MATAGORDA CO., TX', 'TX', 'MATAGORDA', 'STX', 'R', 'WHD')
('73413', '2020-08-21 00:00:00', '143338', '143338', '185962', 'TX EAST TRAN', 'TENNESSEE GAS P/L - HANGING ROCK LATERAL', 'OH', 'SCIOTO', '2', 'R', 'INT')
('73413', '2020-08-22 00:00:00', '143338', '143338', '180962', 'TX EAST TRAN', 'TENNESSEE GAS P/L - HANGING ROCK LATERAL', 'OH', 'SCIOTO', '2', 'R', 'INT')
('73413', '2020-08-23 00:00:00', '143338', '143338', '180962', 'TX E

('73664', '2020-08-27 00:00:00', '34847', '34847', '0', 'TX EAST TRAN', 'SAMSON CHARLTON/HILLEGASS, SOMERSET CO., PA', 'PA', 'SOMERSET', '3', 'R', 'WHD')
('73664', '2020-08-28 00:00:00', '34847', '34847', '0', 'TX EAST TRAN', 'SAMSON CHARLTON/HILLEGASS, SOMERSET CO., PA', 'PA', 'SOMERSET', '3', 'R', 'WHD')
('73664', '2020-08-29 00:00:00', '34847', '34847', '0', 'TX EAST TRAN', 'SAMSON CHARLTON/HILLEGASS, SOMERSET CO., PA', 'PA', 'SOMERSET', '3', 'R', 'WHD')
('73664', '2020-08-30 00:00:00', '34847', '34847', '0', 'TX EAST TRAN', 'SAMSON CHARLTON/HILLEGASS, SOMERSET CO., PA', 'PA', 'SOMERSET', '3', 'R', 'WHD')
('73667', '2020-08-21 00:00:00', '226210', '226210', '29676', 'TX EAST TRAN', 'HOT SPRING POWER PLANT, HOT SPRING CO, AR', 'AR', 'HOT SPRING', 'ETX', 'D', 'END')
('73667', '2020-08-22 00:00:00', '226210', '226210', '76270', 'TX EAST TRAN', 'HOT SPRING POWER PLANT, HOT SPRING CO, AR', 'AR', 'HOT SPRING', 'ETX', 'D', 'END')
('73667', '2020-08-23 00:00:00', '226210', '226210', '59352'

('73925', '2020-08-26 00:00:00', '610625', '610625', '0', 'TX EAST TRAN', 'CNX RICH HILL, GREEN CO. PA', 'PA', 'GREENE', '2', 'R', 'WHD')
('73925', '2020-08-27 00:00:00', '610625', '610625', '0', 'TX EAST TRAN', 'CNX RICH HILL, GREEN CO. PA', 'PA', 'GREENE', '2', 'R', 'WHD')
('73925', '2020-08-28 00:00:00', '610625', '610625', '0', 'TX EAST TRAN', 'CNX RICH HILL, GREEN CO. PA', 'PA', 'GREENE', '2', 'R', 'WHD')
('73925', '2020-08-29 00:00:00', '610625', '610625', '0', 'TX EAST TRAN', 'CNX RICH HILL, GREEN CO. PA', 'PA', 'GREENE', '2', 'R', 'WHD')
('73925', '2020-08-30 00:00:00', '610625', '610625', '0', 'TX EAST TRAN', 'CNX RICH HILL, GREEN CO. PA', 'PA', 'GREENE', '2', 'R', 'WHD')
('73927', '2020-08-21 00:00:00', '152550', '152550', '110486', 'TX EAST TRAN', 'KUDO MIDSTREAM-MARTINSVILLE GATHERING-CR 342-NACOGDOCHES CO', 'TX', 'NACOGDOCHES', 'ETX', 'R', 'WHD')
('73927', '2020-08-22 00:00:00', '152550', '152550', '110486', 'TX EAST TRAN', 'KUDO MIDSTREAM-MARTINSVILLE GATHERING-CR 342-NAC

('79153', '2020-08-26 00:00:00', '112000', '112000', '72402', 'TX EAST TRAN', 'HEAD OF HOT SPRING LATERAL', 'AR', 'HOT SPRING', 'ETX', 'R', 'STA')
('79153', '2020-08-27 00:00:00', '112000', '112000', '31658', 'TX EAST TRAN', 'HEAD OF HOT SPRING LATERAL', 'AR', 'HOT SPRING', 'ETX', 'R', 'STA')
('79153', '2020-08-28 00:00:00', '112000', '112000', '0', 'TX EAST TRAN', 'HEAD OF HOT SPRING LATERAL', 'AR', 'HOT SPRING', 'ETX', 'R', 'STA')
('79153', '2020-08-29 00:00:00', '112000', '112000', '0', 'TX EAST TRAN', 'HEAD OF HOT SPRING LATERAL', 'AR', 'HOT SPRING', 'ETX', 'R', 'STA')
('79153', '2020-08-30 00:00:00', '112000', '112000', '0', 'TX EAST TRAN', 'HEAD OF HOT SPRING LATERAL', 'AR', 'HOT SPRING', 'ETX', 'R', 'STA')
('79660', '2020-08-21 00:00:00', '1857746', '1857746', '340769', 'TX EAST TRAN', 'MARIETTA CUSTODY TRANSFER POINT', 'PA', 'LANCASTER', '3', 'D', 'STA')
('79660', '2020-08-22 00:00:00', '1857746', '1857746', '377401', 'TX EAST TRAN', 'MARIETTA CUSTODY TRANSFER POINT', 'PA', 'LA